# TODO

 - add "index" value to dicts in YAML lists
 - flatten concept/entity information to be inside of main text

In [1]:
from tqdm import tqdm
import os
from glob import glob
import yaml
import json
import re

BASE_DIR = ".."
OUT_DIR = f"{BASE_DIR}/forKinsukAndSjors"


In [2]:
eng = glob(f"{BASE_DIR}/*/English/*.yml")
dutch = glob(f"{BASE_DIR}/*/Dutch/*.yml")
top = glob(f"{BASE_DIR}/TopLevel/*.yml")

yaml_files = top + dutch + eng

yaml_files

['../niveau1/Dutch/HowToUseTheDataHub_20240424.yml',
 '../niveau1/Dutch/DoingResearch_20240425.yml',
 '../niveau1/Dutch/Reporting_20240501.yml',
 '../niveau1/Dutch/Sources_20240425.yml',
 '../niveau1/Dutch/SelectAndDelineate_20240425.yml',
 '../niveau3/Dutch/NZG_20240314.yml',
 '../niveau3/Dutch/KKZ_20240313.yml',
 '../niveau3/Dutch/WMLeiden_20240327.yml',
 '../niveau3/Dutch/KVVAK_20240312.yml',
 '../niveau3/Dutch/KunsthandelVanLier_20240313.yml',
 '../niveau3/Dutch/KoninklijkeBazar_20240313.yml',
 '../niveau3/Dutch/PieterPott_20240312.yml',
 '../niveau3/Dutch/WageningenUniversity_20240327.yml',
 '../niveau2/Dutch/CivilServants_20240320.yml',
 '../niveau2/Dutch/ChristianMission_20240326.yml',
 '../niveau2/Dutch/Trade_20240326.yml',
 '../niveau2/Dutch/MilitaryAndNavy_20240326.yml',
 '../niveau0/Dutch/TopLevel_20240606.yml',
 '../niveau1/English/HowToUseTheDataHub_20240424.yml',
 '../niveau1/English/DoingResearch_20240425.yml',
 '../niveau1/English/Reporting_20240501.yml',
 '../niveau1/E

In [30]:
def parse_filename(orig_path, has_path=False):

    path_part = '.+\/' if has_path else ''
    m = re.search(f'{path_part}(.*)_[0-9]+\.yml', orig_path)
    if m:
        return m.group(1)
    raise ValueError(f"{orig_path} couldn't be parsed!")

def parse_filepath(fp):
    *pref, level, lang, fname  = fp.split(os.path.sep)
    return level, lang, parse_filename(fname)


def addlinkFilename(rels):
    for d1 in rels:
        ((n, val),) = d1.items()
        level, lang, name = parse_filepath(val['link'])
        val = val | {"linkFilename": f"{level}/{lang}/{name}_{lang}"}
        yield {n: val}

def addEntityInfotoText(d, lang):
    assert lang in ("English", "Dutch")
    main = d["Main-text"]["content"]
    rlv = d["Relevant data"]

    header = "Name variations" if lang == "English" else "Naamsvarianten"
    name_vars = "\n".join(f" - {v}" for v in rlv['Name variations'])
    main += f"\n### {header}\n{name_vars}\n\n"

    header = "Period of activity" if lang == "English" else "Periode actief"
    period = f"{rlv['Period of activity']['Year of start']} -- {rlv['Period of activity']['Year of end']}"
    main += f"\n### {header}\n{period}\n\n"

    if isinstance(rlv['Identifiers'], list):
        header = "External identifiers" if lang == "English" else "Externe identificatie"
        external = "\n".join(f" - {v}" for v in rlv['Identifiers'])
        main += f"\n### {header}\n{external}\n\n"
    # d["Main-text"]["content"] = main
    
    return main

In [32]:
from datetime import datetime

for f in tqdm(yaml_files):
    try:
        print(f)
        with open(f) as handle:
            yaml_content = yaml.safe_load(handle)

        level, lang, name = parse_filepath(f)
        yaml_content["File name"] = name
        # link_list = "RelatedAides" if "RelatedAides" in yaml_content else 
        if "RelatedAides" in yaml_content:
            yaml_content["RelatedAides"] = list(addlinkFilename(yaml_content["RelatedAides"]))

        # if "Relevant data" in yaml_content: # and "Main-text" in yaml_content
        #     d["Main-text"]["content"] = addEntityInfotoText(yaml_content, lang)

        
        new_name = f"{OUT_DIR}/{level}/{lang}/{name}_{lang}.json"
        print(new_name)
        print()
    
        # new_name = f.split(os.path.sep, maxsplit=1)[1]
        # new_name = new_name.rsplit(os.path.sep, maxsplit=1)[0]
        # pref, lang = new_name.
        # new_name = f"{OUT_DIR}/{new_name}/{yaml_content['File name']}.json"
        # new_name = new_name.replace(".yml", ".json")
        # new_name = f"{OUT_DIR}/{new_name}"
    
        # os.makedirs(os.path.dirname(new_name), exist_ok=True)
        # with open(new_name, "w") as handle:
        #     json.dump(yaml_content, handle)
    # except yaml.scanner.ScannerError:
    #     print(f)
    # except yaml.parser.ParserError:
    #     print(f)
    except ValueError:
        print(f"{f}'s filename can't be parsed")

 14%|██████████▉                                                                    | 5/36 [00:00<00:00, 183.68it/s]

../niveau1/Dutch/HowToUseTheDataHub_20240424.yml
../forKinsukAndSjors/niveau1/Dutch/HowToUseTheDataHub_Dutch.json

../niveau1/Dutch/DoingResearch_20240425.yml
../forKinsukAndSjors/niveau1/Dutch/DoingResearch_Dutch.json

../niveau1/Dutch/Reporting_20240501.yml
../forKinsukAndSjors/niveau1/Dutch/Reporting_Dutch.json

../niveau1/Dutch/Sources_20240425.yml
../forKinsukAndSjors/niveau1/Dutch/Sources_Dutch.json

../niveau1/Dutch/SelectAndDelineate_20240425.yml
../forKinsukAndSjors/niveau1/Dutch/SelectAndDelineate_Dutch.json

../niveau3/Dutch/NZG_20240314.yml


TypeError: list indices must be integers or slices, not str

In [33]:
f = "../niveau3/Dutch/NZG_20240314.yml"

with open(f) as handle:
    yaml_content = yaml.safe_load(handle)
    level, lang, name = parse_filepath(f)
    yaml_content["File name"] = name
        # link_list = "RelatedAides" if "RelatedAides" in yaml_content else 
    if "RelatedAides" in yaml_content:
        yaml_content["RelatedAides"] = list(addlinkFilename(yaml_content["RelatedAides"]))
        
for k, v in yaml_content.items():
    if isinstance(v, dict):
        print(k)



Relevant data
Sources


---

In [27]:
yaml_content["Relevant data"]["Identifiers"]

'https://www.wikidata.org/entity/Q65850179'

---

# testing

In [ ]:
# are filenames the same across Dutch and English?

def filename(f):
    return f.rsplit(os.path.sep, maxsplit=1)[1]

for d in glob("../forKinsukAndSjors/*"):
    nl = map(filename, glob(d+"/Dutch/*"))
    en = map(filename, glob(d+"/English/*"))
    diff = set(nl) ^ set(en)
    
    if diff:
        print(d, diff)


---

In [21]:
from tqdm import tqdm
import os
from glob import glob
import yaml
import json
import re

BASE_DIR = ".."
OUT_DIR = f"{BASE_DIR}/forKinsukAndSjors"



# eng = glob(f"{BASE_DIR}/*/English/*.yml")
# dutch = glob(f"{BASE_DIR}/*/Dutch/*.yml")
top = glob(f"{BASE_DIR}/TopLevel/*.yml")

yaml_files = top + dutch + eng


def parse_filename(orig_path, has_path=False):

    path_part = '.+\/' if has_path else ''
    m = re.search(f'{path_part}(.*)_[0-9]+\.yml', orig_path)
    if m:
        return m.group(1)
    raise ValueError(f"{orig_path} couldn't be parsed!")

def parse_filepath(fp):
    *pref, level, lang, fname  = fp.split(os.path.sep)
    return level, lang, parse_filename(fname)


def addlinkFilename(rels, lang):
    sub = "Subtopics" if lang == "English" else "Deelonderwerpen"
    for d1 in rels:
        ((n, val),) = d1.items()
        level, lang, name = parse_filepath(val['link'])
        val = val | {"linkFilename": f"{level}/{lang}/{name}_{lang}"}
        if sub in val:
            val[sub] = list(addlinkFilename(val[sub], lang=lang)) 
        
        yield {n: val}

# def addlinkFilename_recurse(



def addEntityInfotoText(d, lang):
    assert lang in ("English", "Dutch")
    main = d["Main-text"]["content"]
    rlv = d["Relevant data"]

    header = "Name variations" if lang == "English" else "Naamsvarianten"
    name_vars = "\n".join(f" - {v}" for v in rlv['Name variations'])
    main += f"\n### {header}\n{name_vars}\n\n"

    header = "Period of activity" if lang == "English" else "Periode actief"
    period = f"{rlv['Period of activity']['Year of start']} -- {rlv['Period of activity']['Year of end']}"
    main += f"\n### {header}\n{period}\n\n"

    if isinstance(rlv['Identifiers'], list):
        header = "External identifiers" if lang == "English" else "Externe identificatie"
        external = "\n".join(f" - {v}" for v in rlv['Identifiers'])
        main += f"\n### {header}\n{external}\n\n"
    # d["Main-text"]["content"] = main
    
    return main



# MAIN


for f in tqdm(yaml_files):
    print(f"processing {f}...")
    try:
        with open(f) as handle:
            yaml_content = yaml.safe_load(handle)
    
        level, lang, name = parse_filepath(f)
        yaml_content["File name"] = name
        # link_list = "RelatedAides" if "RelatedAides" in yaml_content else "Breakdown"
        # yaml_content[link_list] = list(addlinkFilename(yaml_content[link_list], lang=lang))
        
        if "RelatedAides" in yaml_content:
            yaml_content["RelatedAides"] = list(addlinkFilename(yaml_content["RelatedAides"], lang=lang))
        if "Breakdown" in yaml_content:
            
            yaml_content["Breakdown"] = {subtitle: list(addlinkFilename(sublist, lang=lang))
                                         for subtitle, sublist in yaml_content["Breakdown"].items()}


 11%|████▍                                   | 4/36 [00:00<00:00, 37.13it/s]

processing ../niveau1/Dutch/HowToUseTheDataHub_20240424.yml...
processing ../niveau1/Dutch/DoingResearch_20240425.yml...
processing ../niveau1/Dutch/Reporting_20240501.yml...
processing ../niveau1/Dutch/Sources_20240425.yml...
processing ../niveau1/Dutch/SelectAndDelineate_20240425.yml...
processing ../niveau3/Dutch/NZG_20240314.yml...
processing ../niveau3/Dutch/KKZ_20240313.yml...


 31%|███████████▉                           | 11/36 [00:00<00:00, 25.68it/s]

processing ../niveau3/Dutch/WMLeiden_20240327.yml...
processing ../niveau3/Dutch/KVVAK_20240312.yml...
processing ../niveau3/Dutch/KunsthandelVanLier_20240313.yml...
processing ../niveau3/Dutch/KoninklijkeBazar_20240313.yml...
processing ../niveau3/Dutch/PieterPott_20240312.yml...
processing ../niveau3/Dutch/WageningenUniversity_20240327.yml...


 42%|████████████████▎                      | 15/36 [00:00<00:00, 26.39it/s]

...

processing ../niveau3/English/KunsthandelVanLier_20240507.yml...
processing ../niveau3/English/KVVAK_20240508.yml...
processing ../niveau3/English/KoninklijkeBazar_2040503.yml...
processing ../niveau3/English/PieterPott_20240508.yml...


 92%|███████████████████████████████████▊   | 33/36 [00:01<00:00, 22.78it/s]

processing ../niveau3/English/KKZ_20240417.yml...
processing ../niveau3/English/WMLeiden_20240508.yml...
processing ../niveau2/English/Trade_20240316.yml...
processing ../niveau2/English/CivilServants_20240316.yml...
processing ../niveau2/English/MilitaryAndNavy_20240417.yml...


100%|███████████████████████████████████████| 36/36 [00:01<00:00, 24.40it/s]

processing ../niveau2/English/ChristianMission_20240417.yml...
processing ../niveau0/English/TopLevel_20240606.yml...


In [22]:
yaml_content

{'Level': 0,
 'Title': 'Digital research guide',
 'Subtitle': 'Research aides for conducting provenance research into colonial collections',
 'Content': {'content': 'On this page you find various research aides that can assist you in conducting (provenance) research into Dutch colonial collections. The research aides are intended to guide users in the first steps of their research into objects in Dutch collections acquired in a colonial context. The research aides are linked in an intuitive way, allowing relevant information to be quickly found. There are three different types of research aides. Firstly, there are general research aides that answer questions such as *How can I use the data hub for my research?* *Where can I find sources for my research?* and *How do I write a provenance report?* Secondly, there are research aides that provide information on general topics related to colonial collections in the Netherlands, such as the role of civil servants, missionaries and military p